In [7]:
import pandas as pd
import requests
import json

In [2]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## Criando arquivo automaticamente

In [4]:
pd.DataFrame(
    [
        {"UserID": 4},
        {"UserID": 5},
        {"UserID": 6}
    ]
).to_csv('./SDW2023.csv', index=False)

# **Extract**

## Leitura do arquivo

In [5]:
df = pd.read_csv('./SDW2023.csv')
df.head()

,UserID
0,4
1,5
2,6


In [6]:
user_ids = df['UserID'].tolist()
user_ids

[4, 5, 6]

## Resgatando informações dos usuários na api utilizando **UserID**

In [14]:
def get_user(id):

  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

In [ ]:
users_info = [ user for user_id in user_ids if (user := get_user(user_id)) is not None ]
users_info

# **Transform**

In [ ]:
!pip install openai

In [28]:
openai_api_key = 'your-api-key-here'

In [29]:
import openai

openai.api_key = openai_api_key

In [ ]:
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users_info:
  news = generate_ai_news(user)
  print(news)

# **Load**

In [33]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users_info:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}")

User Pip updated? True
User Pep updated? True
